<a href="https://colab.research.google.com/github/ancestor9/24_fall_Big-Data-Processing-coding-/blob/main/1017_DB_02_Sqliite_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **데이터베이스 이해**
### **1. Sqlite 이해하기**
#### **[Dbeaver 설치하고 Sqlite DB 확인](https://dbeaver.io/)**
### **2. Data flows like stream with Buffer**
### **3. WIth statement (Context Manager) 이해**

In [ ]:
! pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.3 MB/s eta 0:00:00


In [ ]:
import sqlite3
import pandas as pd
from faker import Faker
import random

# Initialize Faker instance
fake = Faker()

# Connect to SQLite (creates the database file)
conn = sqlite3.connect('flight_booking.db')
cursor = conn.cursor()

# 인스턴스의 개수
sample_siz= 1000


In [ ]:
# Ensure all tables are created before inserting data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Users (
        Email TEXT PRIMARY KEY
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Passengers (
        ID INTEGER PRIMARY KEY,
        Name TEXT,
        Email TEXT,
        Gender TEXT,
        Age INTEGER,
        FOREIGN KEY(Email) REFERENCES Users(Email)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Bookings (
        ID INTEGER PRIMARY KEY,
        User_Email TEXT,
        Booking_Date TEXT,
        FOREIGN KEY(User_Email) REFERENCES Users(Email)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Flights (
        Code TEXT PRIMARY KEY
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Journeys (
        ID INTEGER PRIMARY KEY,
        Flight_Code TEXT,
        Date_Of_Journey TEXT,
        Booking_ID INTEGER,
        FOREIGN KEY(Flight_Code) REFERENCES Flights(Code),
        FOREIGN KEY(Booking_ID) REFERENCES Bookings(ID)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS BP_Relationships (
        Booking_ID INTEGER,
        Passenger_ID INTEGER,
        FOREIGN KEY(Booking_ID) REFERENCES Bookings(ID),
        FOREIGN KEY(Passenger_ID) REFERENCES Passengers(ID)
    )
''')


In [ ]:

# Function to generate fake users
def generate_fake_users(n=sample_siz):
    users = []
    for _ in range(n):
        users.append((fake.email(),))
    return users

# Function to generate fake passengers
def generate_fake_passengers(users, n=sample_siz):
    passengers = []
    for i in range(n):
        name = fake.name()
        email = random.choice(users)[0]  # Randomly select from generated users
        gender = random.choice(['Male', 'Female'])
        age = random.randint(18, 80)
        passengers.append((i + 1, name, email, gender, age))
    return passengers

# Function to generate fake bookings
def generate_fake_bookings(users, n=sample_siz):
    bookings = []
    for i in range(n):
        user_email = random.choice(users)[0]
        booking_date = fake.date_this_year()
        bookings.append((i + 1, user_email, booking_date))
    return bookings

# Function to generate fake flights
def generate_fake_flights(n=sample_siz):
    flights = []
    for _ in range(n):
        flight_code = fake.bothify('FL###')  # Generate flight code like FL123
        flights.append((flight_code,))
    return flights

# Function to generate fake journeys
def generate_fake_journeys(bookings, flights, n=sample_siz):
    journeys = []
    for i in range(n):
        flight_code = random.choice(flights)[0]
        date_of_journey = fake.date_this_year()
        booking_id = random.choice(bookings)[0]
        journeys.append((i + 1, flight_code, date_of_journey, booking_id))
    return journeys

# Function to generate fake BP relationships
def generate_fake_bp_relationships(bookings, passengers, n=sample_siz):
    bp_relationships = []
    for _ in range(n):
        booking_id = random.choice(bookings)[0]
        passenger_id = random.choice(passengers)[0]
        bp_relationships.append((booking_id, passenger_id))
    return bp_relationships


In [ ]:
# Generate fake data
users_data = generate_fake_users(sample_siz)
passengers_data = generate_fake_passengers(users_data, sample_siz)
bookings_data = generate_fake_bookings(users_data, sample_siz)
flights_data = generate_fake_flights(sample_siz)
journeys_data = generate_fake_journeys(bookings_data, flights_data, sample_siz)
bp_relationships_data = generate_fake_bp_relationships(bookings_data, passengers_data, sample_siz)

# Insert data into SQLite tables
cursor.executemany('INSERT OR IGNORE INTO Users (Email) VALUES (?)', users_data)
cursor.executemany('INSERT OR IGNORE INTO Passengers VALUES (?, ?, ?, ?, ?)', passengers_data)
cursor.executemany('INSERT OR IGNORE INTO Bookings VALUES (?, ?, ?)', bookings_data)
cursor.executemany('INSERT OR IGNORE INTO Flights (Code) VALUES (?)', flights_data)
cursor.executemany('INSERT OR IGNORE INTO Journeys VALUES (?, ?, ?, ?)', journeys_data)
cursor.executemany('INSERT OR IGNORE INTO BP_Relationships VALUES (?, ?)', bp_relationships_data)


# Commit changes
conn.commit()


In [ ]:

# Load tables into pandas DataFrames
df_users = pd.read_sql_query('SELECT * FROM Users', conn)
df_passengers = pd.read_sql_query('SELECT * FROM Passengers', conn)
df_bookings = pd.read_sql_query('SELECT * FROM Bookings', conn)
df_flights = pd.read_sql_query('SELECT * FROM Flights', conn)
df_journeys = pd.read_sql_query('SELECT * FROM Journeys', conn)
df_bp_relationships = pd.read_sql_query('SELECT * FROM BP_Relationships', conn)

# Store data as dictionary
dataframes_dict = {
    'Users': df_users,
    'Passengers': df_passengers,
    'Bookings': df_bookings,
    'Flights': df_flights,
    'Journeys': df_journeys,
    'BP_Relationships': df_bp_relationships
}

# Close connection
conn.close()


In [ ]:
for table_name, df in dataframes_dict.items():
  print(f"--- {table_name} ---")
  display(df.head())

--- Users ---


,Email
0,cookaustin@example.com
1,jeffreystephens@example.org
2,kevin25@example.org
3,jeffrey00@example.net
4,obender@example.com


--- Passengers ---


,ID,Name,Email,Gender,Age
0,1,Timothy Hale,matthew76@example.org,Female,22
1,2,Jennifer Patterson,kellywheeler@example.com,Female,70
2,3,Phyllis Ball,michael78@example.net,Female,32
3,4,Rhonda Hill,vbarnett@example.com,Male,39
4,5,Joseph Prince,rmaxwell@example.net,Male,21


--- Bookings ---


...

,Code
0,FL569
1,FL983
2,FL797
3,FL229
4,FL644


--- Journeys ---


,ID,Flight_Code,Date_Of_Journey,Booking_ID
0,1,FL434,2024-04-19,966
1,2,FL487,2024-02-13,178
2,3,FL226,2024-01-22,934
3,4,FL453,2024-04-22,225
4,5,FL411,2024-06-06,597


--- BP_Relationships ---


,Booking_ID,Passenger_ID
0,978,166
1,368,613
2,307,526
3,447,605
4,201,678


In [ ]:
import pandas as pd

# Perform joins to combine all data into one DataFrame
df_combined = df_bp_relationships \
    .merge(df_bookings, left_on='Booking_ID', right_on='ID', suffixes=('_bp', '_booking')) \
    .merge(df_passengers, left_on='Passenger_ID', right_on='ID', suffixes=('_booking', '_passenger')) \
    .merge(df_users, left_on='Email', right_on='Email') \
    .merge(df_journeys, left_on='Booking_ID', right_on='Booking_ID') \
    .merge(df_flights, left_on='Flight_Code', right_on='Code')

# Remove duplicate columns if needed
df_combined.drop(columns=['ID_booking', 'ID_passenger', 'Code'], inplace=True)


In [ ]:
df_combined

,Booking_ID,Passenger_ID,User_Email,Booking_Date,Name,Email,Gender,Age,ID,Flight_Code,Date_Of_Journey
0,978,166,odean@example.org,2024-04-30,Julie Cochran,kyle52@example.net,Male,38,213,FL815,2024-04-28
1,978,166,odean@example.org,2024-04-30,Julie Cochran,kyle52@example.net,Male,38,969,FL525,2024-01-13
2,368,613,gary12@example.com,2024-01-01,Christopher Phillips,eugene37@example.org,Male,72,607,FL366,2024-05-06
3,368,613,gary12@example.com,2024-01-01,Christopher Phillips,eugene37@example.org,Male,72,793,FL478,2024-01-21
4,307,526,vlong@example.org,2024-07-02,Mark Booker,yboone@example.org,Female,70,137,FL033,2024-01-29
...,...,...,...,...,...,...,...,...,...,...,...
962,404,701,marciasheppard@example.net,2024-01-23,Corey Adams,douglashodge@example.org,Male,23,107,FL122,2024-06-04
963,404,701,marciasheppard@example.net,2024-01-23,Corey Adams,douglashodge@example.org,Male,23,261,FL991,2024-03-04
964,95,281,lorisoto@example.com,2024-04-16,Jodi Morales,michellechung@example.com,Male,51,696,FL482,2024-02-04
965,292,597,noah87@example.net,2024-07-30,Lauren Salinas,emma92@example.com,Male,48,891,FL234,2024-07-11


In [ ]:
pd.read_sql_query('SELECT * FROM Users', conn)

ProgrammingError: Cannot operate on a closed database.

In [ ]:
conn = sqlite3.connect('flight_booking.db')
cursor = conn.cursor()

In [ ]:
pd.read_sql_query('SELECT * FROM Users', conn)

,Email
0,cookaustin@example.com
1,jeffreystephens@example.org
2,kevin25@example.org
3,jeffrey00@example.net
4,obender@example.com
...,...
995,melissa54@example.org
996,elizabeth22@example.org
997,kmiles@example.net
998,beckerdenise@example.com


In [ ]:
conn.close()

## Cntext Manager
### With function ~ 구문

In [ ]:
# prompt: with 구문으로 읽기

import sqlite3
import pandas as pd

conn = sqlite3.connect('flight_booking.db')

with conn:
  df_users = pd.read_sql_query('SELECT * FROM Users', conn)
  df_passengers = pd.read_sql_query('SELECT * FROM Passengers', conn)
  df_bookings = pd.read_sql_query('SELECT * FROM Bookings', conn)
  df_flights = pd.read_sql_query('SELECT * FROM Flights', conn)
  df_journeys = pd.read_sql_query('SELECT * FROM Journeys', conn)
  df_bp_relationships = pd.read_sql_query('SELECT * FROM BP_Relationships', conn)

pd.read_sql_query('SELECT * FROM Users', conn)

,Email
0,cookaustin@example.com
1,jeffreystephens@example.org
2,kevin25@example.org
3,jeffrey00@example.net
4,obender@example.com
...,...
995,melissa54@example.org
996,elizabeth22@example.org
997,kmiles@example.net
998,beckerdenise@example.com


In [ ]:
pd.read_sql_query('SELECT * FROM Passengers', conn)

,ID,Name,Email,Gender,Age
0,1,Timothy Hale,matthew76@example.org,Female,22
1,2,Jennifer Patterson,kellywheeler@example.com,Female,70
2,3,Phyllis Ball,michael78@example.net,Female,32
3,4,Rhonda Hill,vbarnett@example.com,Male,39
4,5,Joseph Prince,rmaxwell@example.net,Male,21
...,...,...,...,...,...
995,996,Christopher Welch,angelagriffin@example.org,Male,60
996,997,David Harris,johnhill@example.com,Male,32
997,998,Christina Sanchez,qmurray@example.com,Male,71
998,999,David Hamilton,kellyperez@example.org,Female,39


In [ ]:
pd.read_sql_query('SELECT * FROM Passengers', conn)

,ID,Name,Email,Gender,Age
0,1,Timothy Hale,matthew76@example.org,Female,22
1,2,Jennifer Patterson,kellywheeler@example.com,Female,70
2,3,Phyllis Ball,michael78@example.net,Female,32
3,4,Rhonda Hill,vbarnett@example.com,Male,39
4,5,Joseph Prince,rmaxwell@example.net,Male,21
...,...,...,...,...,...
995,996,Christopher Welch,angelagriffin@example.org,Male,60
996,997,David Harris,johnhill@example.com,Male,32
997,998,Christina Sanchez,qmurray@example.com,Male,71
998,999,David Hamilton,kellyperez@example.org,Female,39


In [ ]:
conn.close()

In [ ]:
with sqlite3.connect('flight_booking.db') as conn:
  df_users = pd.read_sql_query('SELECT * FROM Users', conn)
  display(df_users)

,Email
0,cookaustin@example.com
1,jeffreystephens@example.org
2,kevin25@example.org
3,jeffrey00@example.net
4,obender@example.com
...,...
995,melissa54@example.org
996,elizabeth22@example.org
997,kmiles@example.net
998,beckerdenise@example.com


In [ ]:
pd.read_sql_query('SELECT * FROM Passengers')

TypeError: read_sql_query() missing 1 required positional argument: 'con'

In [ ]:
pd.read_sql_query('SELECT * FROM Passengers', conn)

,ID,Name,Email,Gender,Age
0,1,Timothy Hale,matthew76@example.org,Female,22
1,2,Jennifer Patterson,kellywheeler@example.com,Female,70
2,3,Phyllis Ball,michael78@example.net,Female,32
3,4,Rhonda Hill,vbarnett@example.com,Male,39
4,5,Joseph Prince,rmaxwell@example.net,Male,21
...,...,...,...,...,...
995,996,Christopher Welch,angelagriffin@example.org,Male,60
996,997,David Harris,johnhill@example.com,Male,32
997,998,Christina Sanchez,qmurray@example.com,Male,71
998,999,David Hamilton,kellyperez@example.org,Female,39
